In [2]:
import sys
import os

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))  # Adjust the path accordingly

In [3]:
! pip install -U nltk
! pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
from load_preprocessed_data import load_ladino_pos

weak_dataset_file_path = '../data/weak/ladino-pos.txt'
weak_tags, weak_tags_dict = load_ladino_pos(weak_dataset_file_path) # import ladino tokens into custom data definition

../data/weak/ladino-pos.txt


In [5]:
# check it worked
for item in weak_tags[0]:
    print(item)

¡ (PUNCT)
Venidos (NOUN)
buenos (ADJ)
! (PUNCT)


In [6]:
from datasets import load_dataset

labeled_data = weak_tags
unlabeled_data = load_dataset("collectivat/salom-ladino-articles")['train']
train = unlabeled_data[:10300]['text']
validate = unlabeled_data[10301:10500]['text']
test = unlabeled_data[10501:10685]['text']

/home/ezgibas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import spacy

nlp = spacy.load("es_core_news_sm")
weakly_labeled = []

for sentence in train:
    doc = nlp(sentence)
    tagged_sentence = [(token.text, token.pos_) for token in doc]
    weakly_labeled.append(tagged_sentence)


In [8]:
import nltk

def laplace_smoothing(freq_dist, bins):
    return nltk.LaplaceProbDist(freq_dist, bins)

def trainer(labeled, unlabeled):
    trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()
    hmm_tagger = trainer.train_supervised(labeled, estimator=laplace_smoothing)
    combined_data = labeled + unlabeled
    hmm_tagger = trainer.train_supervised(combined_data, estimator=laplace_smoothing)
    return hmm_tagger

# format_unlabeled_data = [data.split() for data in train]
format_labeled_data = [[(token.get_word(), token.get_pos()) for token in sentence] for sentence in labeled_data]

hmm_tagger = trainer(format_labeled_data, weakly_labeled)

In [9]:
# save the hmm so we don't have to train every time
import pickle

# Save the trained HMM model to a file
with open("../results/hmm_tagger.pkl", "wb") as f:
    pickle.dump(hmm_tagger, f)

In [ ]:
from preprocessing import tokenize

sentence = test[100]
tokens, _ = tokenize(sentence)

tagged_sentence = hmm_tagger.tag(tokens)
print(tagged_sentence)

# accuracy = hmm_tagger.accuracy(formatted_sentences)
# print(f"Accuracy: {accuracy:.2%}")

[('Ma', 'PROPN'), ('yo', 'PRON'), ('no', 'ADV'), ('se', 'PRON'), ('meldar', 'VERB'), ('!', 'PUNCT')]
